# CPSC 477/577 Spring 2022, HW4 Parts 3 & 4: Pretraining and fine-tuning transformers with Huggingface and OpenAI

Due: April 27th, 11:59pm

*TF: Malcolm Sailor*

## Please write your name and NetID below

Name: Olina (Qiuyu) Zhu 

NetID: qz258 

# Part 3 Pretraining and Fine-tuning with Huggingface

In this part, we will use pre-trained machine translation models provided by Huggingface.

First, run the following cell to install the libraries we will need for this notebook, and define the Placeholder object we will use below.

In [1]:
!pip install -qq datasets transformers[sentencepiece] sacrebleu openai

# If you get the following error message, you can ignore it:
# ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
# datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.

class Placeholder:
    @property
    def DO(self):
        raise NotImplementedError("You haven't yet implemented this part of the assignment yet")

TO = Placeholder()

     |████████████████████████████████| 325 kB 11.2 MB/s 
     |████████████████████████████████| 4.0 MB 39.8 MB/s 
     |████████████████████████████████| 90 kB 10.7 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 212 kB 41.7 MB/s 
     |████████████████████████████████| 136 kB 49.0 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 1.1 MB 48.5 MB/s 
     |████████████████████████████████| 127 kB 51.1 MB/s 
     |████████████████████████████████| 162 kB 46.5 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 271 kB 50.6 MB/s 
     |████████████████████████████████| 144 kB 54.7 MB/s 
     |████████████████████████████████| 895 kB 46.6 MB/s 
     |████████████████████████████████| 6.6 MB 45.6 MB/s 
     |█████

## Part 3.1 Using a pre-trained model.

Huggingface's `pipeline()` function provides a convenient way of fetching pretrained models for a wider variety of tasks. (For more info, see the tutorial [here](https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/chapter1/section3.ipynb).) 

To use `pipeline()`, we just give it a task (here "translation") and (optionally) the name of a model checkpoint. (If we omit the model checkpoint, it will choose the default model for the given task.) It returns an object that bundles a model together with its preprocessing and postprocessing steps in a single pipeline.

In the following cell, we initialize a pipeline using the ["Helsinki-NLP/opus-mt-de-en"](https://huggingface.co/Helsinki-NLP/opus-mt-de-en) [Marian](https://huggingface.co/docs/transformers/model_doc/marian) model, the model we will be using throughout this section of the assignment.

In [2]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-de-en"
translator = pipeline("translation", model=model_checkpoint)


Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/284M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/750k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Let's have it translate the first sentence of Goethe's *Faust* into English.

In [3]:
translator("Ihr naht euch wieder, schwankende Gestalten!")

[{'translation_text': "You're approaching again, fluctuating figures!"}]

Your turn! This assignment focuses on machine translation, but it is easy to use `pipeline` for other tasks too. Go to the Huggingface [Model hub](https://huggingface.co/models). Find a model that supports text generation. Then use `pipeline` with the `"text-generation"` task and the model in question to complete the prompt

> "The rest of this assignment will be"

In [4]:
# STUDENT 
model_textgen = "distilgpt2"
generator = pipeline("text-generation", model=model_textgen)
generator("The rest of this assignment will be")

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The rest of this assignment will be provided on the basis of results.'}]

## Part 3.2 Fine-tuning a model.

Returning to machine translation, now we are going to try fine-tuning a pretrained model. For this task, we will use the [KDE4 dataset](https://huggingface.co/datasets/kde4). This dataset consists of texts associated with the KDE Linux apps. For this reason, it likely contains more computer and programming related terms than the pretrained model may be expecting. Fine-tuning may therefore be useful.

Use the [load_dataset](https://huggingface.co/docs/datasets/master/en/package_reference/loading_methods#datasets.load_dataset) function to load the dataset. As always in this assignment, we will translate from German to English. 

In [5]:
from datasets import load_dataset

raw_datasets = load_dataset("kde4", lang1="de", lang2="en") # STUDENT 

Using custom data configuration de-en-lang1=de,lang2=en


Generating train split: 0 examples [00:00, ? examples/s]

Dataset kde4 downloaded and prepared to /root/.cache/huggingface/datasets/kde4/de-en-lang1=de,lang2=en/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Inspect the resulting dataset, observe that it has only a single "train" split.

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 224035
    })
})

Use the [`Dataset.train_test_split()`](https://huggingface.co/docs/datasets/master/en/package_reference/main_classes#datasets.Dataset.train_test_split) method to create a "validation" split containing exactly 3000 examples. The rest of the examples should be in the training set.

By default, the split names will be "train" and "test", so you'll have to rename the "test" set. The `Dataset.pop()` method might be useful for this purpose. 

In [7]:
# STUDENT 
perc_val = 3000/101593
raw_datasets = raw_datasets['train'].train_test_split(test_size=perc_val)
raw_datasets["validation"] = raw_datasets.pop("test") 

Inspect `raw_datasets` again to ensure it now has the appropriate splits.

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 217419
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 6616
    })
})

# Data preprocessing

As we have seen, modern NLP models use various subword tokenization schemes in order to tokenize text. To get the correct tokenizer to use with the model we will use, we can use the `AutoTokenizer` class.

We will use the 

In [9]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-de-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="tf")

Let's inspect what the tokenizer does with some German and English text. Here's a quote attributed to J.S. Bach, together with its translation into English.

In [10]:
de_sentence = "Sie müssen nur die richtige Taste im richtigen Moment berühren, und das Instrument selbst ertönt"
en_sentence = "All one has to do is hit the right keys at the right time and the instrument plays itself."

If you run the following cell, you can see that the tokenizer treats all of the German words as tokens, with the exception of the verb "ertönt" which it separates into the head "er" and the suffix "tönt". Looks good!

In [11]:
print(" ".join(tokenizer.tokenize(de_sentence)))

▁Sie ▁müssen ▁nur ▁die ▁richtige ▁Taste ▁im ▁richtigen ▁Moment ▁berühren , ▁und ▁das ▁Instrument ▁selbst ▁er tönt


But if you run the following code cell, the tokenizer doesn't seem to know many of the English words, including common words like "one" and "has".

**In the following text cell, explain what has gone wrong.**

In [12]:
print(" ".join(tokenizer.tokenize(en_sentence)))

▁All ▁on e ▁ha s ▁to ▁do ▁is ▁h it ▁the ▁ right ▁ key s ▁at ▁the ▁ right ▁ time ▁and ▁the ▁ instrument ▁ play s ▁it s elf .


[Your answer goes here] # STUDENT 
<br> **Response:** This potentially has to do with the fact that the autotokenizer was first used on German training text, hence the tokenizer is set to a German tokenizer based on the model, which does not work well with English text. 

For tokenizing English, we need to use the `as_target_tokenizer()` context manager as follows. The output should now look much better!

In [13]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.tokenize(en_sentence))

['▁All', '▁one', '▁has', '▁to', '▁do', '▁is', '▁hit', '▁the', '▁right', '▁keys', '▁at', '▁the', '▁right', '▁time', '▁and', '▁the', '▁instrument', '▁plays', '▁itself', '.']


We will use the `Dataset.map()` method to tokenize our dataset. The main argument to this method is a callable, which we are calling `preprocess_function`. This callable will receive a dictionary as its argument. The keys are the same as the features of the dataset (in this case, "id" and "translation").

In [14]:
max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    inputs = [ex["de"] for ex in examples["translation"]]
    targets = [ex["en"] for ex in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True) # STUDENT TODO 
    with tokenizer.as_target_tokenizer(): # STUDENT 
        labels = tokenizer(targets, max_length=max_target_length, truncation=True) # STUDENT 

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

proc_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

  0%|          | 0/218 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

Now we need to fetch the actual model to fine-tune. We will use the `AutoModelForSeq2SeqLM` class in order to do so.

In [15]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

We also need a data collator. This will play the same role as the `collate_fn` from the previous notebook, collating examples into a batch and adding padding.

In [16]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Next we instantiate the `Seq2SeqTrainingArguments` with our desired training arguments.

In [18]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"de-en",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    max_steps=2000,
    predict_with_generate=True,
    fp16=True,
    generation_max_length=max_target_length,
)

We also need to do provide a metric for evaluation. We will use SacreBLEU, which is a refinement of the BLEU metric we learned about in class.

You can use the below implementation; you don't need to change anything about it.

In [19]:
import numpy as np

from datasets import load_metric

metric = load_metric("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Sometimes -100s are used as pads in the labels. We will replace them as
    # we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Strip whitespace and put the decoded labels in the expected format 
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

Now, we instantiate the `Seq2SeqTrainer` class which we will call to actually run training and evaluation. You need to work out what the first two arguments should be.

In [20]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer( 
    model = model, # STUDENT 
    args = args, # STUDENT 
    train_dataset=proc_datasets["train"],
    eval_dataset=proc_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend


Let's run the evaluating to see how the pre-trained model does, without fine-tuning. (This should take ~ 3 minutes.) Inspect the output and see what BLEU score you got.

In [ ]:
# took significantly longer than 3 minutes 

In [21]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 6616
  Batch size = 64


{'eval_bleu': 33.60437607306265,
 'eval_loss': 1.82985258102417,
 'eval_runtime': 1025.2617,
 'eval_samples_per_second': 6.453,
 'eval_steps_per_second': 0.101}

Now, let's actually train the model. This is as simple as calling `trainer.train()`.

To save time, we are going to train for only 2000 steps. (This is not even one full epoch, which would be number of examples / batch size = 221035 / 32 = ~6900 steps.) The training should take a little less than 10 minutes.

Note that, if we reduced the size of our training set, we could train for many more epochs in the same number of steps. For example, if we used only 32 * 500 = 16,000 examples, we could train for 4 epochs. **In the following cell, state what you believe would be the advantages or disadvantages of reducing the training set size in order to increase the number of epochs.**

[Your answer here] # STUDENT 
<br> **Response:** Reducing the size of the training set to increase the number of epochs could bring benefits such as decreasing perplexity and loss, but a it also has disadvantages such as potentially acheiving less reduction of loss/perplexity in each epoch. 

In [ ]:
# note trainer took > 1 hr to run

In [22]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 217419
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2000


Step,Training Loss
500,1.714600
1000,1.624600
1500,1.580100
2000,1.577600


Saving model checkpoint to de-en/checkpoint-2000
Configuration saved in de-en/checkpoint-2000/config.json
Model weights saved in de-en/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in de-en/checkpoint-2000/tokenizer_config.json
Special tokens file saved in de-en/checkpoint-2000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2000, training_loss=1.624211639404297, metrics={'train_runtime': 4054.1957, 'train_samples_per_second': 15.786, 'train_steps_per_second': 0.493, 'total_flos': 1365392240934912.0, 'train_loss': 1.624211639404297, 'epoch': 0.29})

Now that we've fine-tuned the model, let's try evaluating it again. Again, this should take about 3 minutes.

Inspect your BLEU score. It should have gone up a little.

Keep in mind that we abbreviated the training in the interest of saving time. If we ran for several full epochs we could expect to see a larger improvement.

In [ ]:
# took significantly longer than 3 mins 
# > 17 mins 
# bleu score increased by ~3 

In [23]:
trainer.evaluate() # STUDENT  

***** Running Evaluation *****
  Num examples = 6616
  Batch size = 64


{'epoch': 0.29,
 'eval_bleu': 36.52019995504327,
 'eval_loss': 1.4945979118347168,
 'eval_runtime': 1057.0316,
 'eval_samples_per_second': 6.259,
 'eval_steps_per_second': 0.098}

# Part 4 GPT-3

In this part, we are going to use the Open AI API to query [GPT-3](https://arxiv.org/abs/2005.14165), a large and powerful language model. We will ask it to perform zero-shot translation. You can also [experiment online](https://beta.openai.com/playground) with GPT-3. (Note that the HuggingFace hub includes [GPT-2](https://huggingface.co/models?other=gpt2), GPT-3's predecessor.)

First, visit [Open AI](https://openai.com/) and set up an account if you don't already have one.

Next, once you are logged in, visit [this page](https://beta.openai.com/account/api-keys) to get your API key. Copy that API key into the following cell, and then run the cell. This will store your API key in an environment variable. 

Then, 

1. **delete your API key from the cell**
2. **clear the cell outputs so your API key is no longer visible**

Do this **immediately** otherwise you are liable to forget! Please do not include your API key when you submit your homework.

In [ ]:
# STUDENT 
%env OPENAI_API_KEY=

The following cell defines a function that will allow you to send a prompt the GPT-3 API and get the response. It returns a tuple consisting first of

1. the text completion
2. the complete response object, in case you wish to query it.

In [25]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def print_gpt3_response(prompt):
    # returns a tuple consisting of the text completion and the full response
    response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt,
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    text = response["choices"][0]["text"].strip()
    print(text)

Now let's try using our `get_gpt3_response()` function to try translating German into English.

Here are a couple German sentences you can try translating. Or you can search for some German text online to translate. Or, if you speak German, you can invent some German text of your own!

- "Das Studium und allgemein das Streben nach Wahrheit und Schönheit ist ein Gebiet, auf dem wir das ganze Leben lang Kinder bleiben dürfen." Einstein.
- "Der Mensch kann zwar tun, was er will, aber er kann nicht wollen, was er will." Schopenhauer.

GPT-3 is an example of an *auto-regressive* model. It is trained to predict the next token at each step of a sentence. Given a prompt, it will attempt to complete the prompt.

Given that this is the case, what should you expect to happen if you provide GPT-3 with a German sentence? Try it, and see if your expectation is confirmed.

Can you think of a way we might get GPT-3 to translate a sentence from German to English? Give it a try with at least one sentence, and display the returned text.

In [32]:
print_gpt3_response("Das Studium und allgemein das Streben nach Wahrheit und Schönheit ist ein Gebiet, auf dem wir das ganze Leben lang Kinder bleiben dürfen.") # STUDENT 

«

Albert Einstein



Anfang der 18. Jahrhundert waren es nur wenige ausgewählte Leute, die Zugang zu Schulen und Universitäten hatten. Die Mehrheit der Menschen konnte nach der Schule nicht weiter studieren. Die Schulen und Universitäten waren auch meistens von Katholiken und Protestanten besucht.



Nach der Aufklärung änderte sich dies. Es entstanden neue Universitäten und Schulen. Diese waren jetzt auch für finanzschwächere Menschen zugänglich.



Nach der Französischen Revolution wurden in Deutschland viele Universitäten gegründet. Die Universitäten waren jetzt auch für Juden und Mitglieder von kleinen Konfessionen geöffnet.



In der Weimarer Republik gab es ganz besondere Universitäten. Es


In [33]:
print_gpt3_response("Der Mensch kann zwar tun, was er will, aber er kann nicht wollen, was er will means in English")

<blockquote>
<p>A person can do what he wants, but he cannot want what he wants.</p>
</blockquote>


Of course, translation is far from the only thing GPT-3 can be used for. Can you find a task that GPT-3 seems to be good at?

In [34]:
print_gpt3_response("Although the weather was fine in the morning, ") # STUDENT 
# good at finishing a sentence logically and giving endings to stories 

the forecast called for rain in the afternoon.


Can you find a task that GPT-3 is bad at (or, at least, at which it doesn't always succeed)?

In [36]:
print_gpt3_response("The exchange rate between USD and RMB today is") # STUDENT 
# bad at predicting technical statements or keeping information updated (current affairs) 
# usd to rmb is 6.55 as of Apr 27, 2022 01:40. 

about 7.3 yuan to the dollar.

7.3 yuan to the dollar.
